In [1]:
import pandas as pd
import numpy as np

from qgrid import show_grid

from siuba import *
from plotnine import *

In [2]:
major_fields_raw = pd.read_excel(
    "./data/sed17-sr-tab012.xlsx",
    skiprows = 3
)


## Preprocessing

In [3]:
renamed = major_fields_raw.rename(columns = {"Field of study": "field"})

melted_majors = renamed.melt(id_vars = "field", var_name = "key")

major_fields = (
    melted_majors
    .assign(
        # note: Series.where replaces values when condition is false or nan
        #       e.g. Series.where(condition, to_replace_if_false_or_nan)
        year= _.key.replace("Unnamed", np.nan, regex = True),
        type=_.value.where(_.value.str.contains("Number|Percent"), np.nan),
        value=lambda d: pd.to_numeric(d.value, "coerce")
    )
    .assign(year=_.year.ffill(), type=_.type.ffill())
    .drop(columns = "key")
    .loc[_.value.notna()]
    # this is spread(_.type, _.value) in pandas --------
    # unstack to row-index: field, type. multi-column-index: value[Number, Percent]
    .pipe(_.set_index(["field", "year", "type"]).unstack(level = -1))
    # get rid of multi-column-index
    .pipe(_.reset_index(col_level = 1).droplevel(None, axis = 1))
    # make it so our columns don't have a "name"
    .pipe(_.rename_axis(columns = None))
)

major_fields

NameError: name 'major_fields' is not defined

In [ ]:
fine_fields = (
    pd.read_excel("./data/sed17-sr-tab013.xlsx", skiprows=3)
    .rename(columns = {"Fine field of study": "field"})
    .melt(id_vars = "field", var_name = "year", value_name = "number")
    # warning: can't index like below if our index has nans
    .loc[_.number != 'na']
    .assign(
        field = _.field.str.strip(),
        year = _.year.astype(float), 
        number = _.number.astype(float)        
    )
)

fine_fields

## Plotting degrees over time for 6 degrees

In [ ]:
# investigated why a space was there
ceramics = fine_fields >> filter(_.field.str.startswith("Ceramic"))
ceramics.values[0]



In [ ]:
from random import sample

#field_subset = sample(list(fine_fields.field.unique()), 6)

field_subset = ['Marine biology and biological oceanography',
 'Educational and instructional technology',
 'Astronomy and astrophysics',
 'Organizational behavior',
 'Literacy and reading educationm',
 'Ceramic sciences engineering']

(
    fine_fields
    .loc[_.field.isin(field_subset)]
    >> ggplot(aes("year", "number", color="field"))
     + geom_line()
)

In [ ]:
broad_fields_raw = pd.read_excel("./data/sed17-sr-tab014.xlsx", skiprows=4)

sex = ["All", "Male", "Female", "All doctorate recipientsa", "All fieldsa"]


broad_fields_formatted = (
    broad_fields_raw
    .rename(columns = {"Unnamed: 0": "field"})
    .loc[~_.field.isin(sex)]
    .assign(
        field=_.field.rename({
            "Life sciencesb": "Life sciences",
            "Otherc": "Other"
        })
    )
)

broad_fields = broad_fields_formatted.field

broad_fields

In [ ]:
recipients_year_field_sex_raw = pd.read_excel("./data/sed17-sr-tab015.xlsx", skiprows=3)
recipients_year_field_sex_raw.columns = recipients_year_field_sex_raw.columns.map(str)


recipients_year_field_sex = (
    recipients_year_field_sex_raw
    .rename(columns = {"Sex and major field of study": "field"})
    .loc[:,~_.columns.str.contains("change")]
    .assign(
        field=_.field.replace({
            "All doctorate recipientsa": "All",
            "Otherb": "Other"
            }),
        sex=_.field.where(_.field.isin(["Male", "Female", "All"])),
        broad_field= _.field.where(_.field.isin(broad_fields))
    )
    .assign(sex = _.sex.ffill(), broad_field = _.broad_field.ffill())
    .melt(id_vars = ["sex", "broad_field", "field"], var_name = "year", value_name = "number")
    .loc[(~_.field.isin(sex)) & _.number.notna()]
    .assign(year = _.year.astype(int))
)

recipients_year_field_sex

In [ ]:
(
    recipients_year_field_sex
    >> filter(_.sex != "All", _.broad_field == "Mathematics and computer sciences")
    >> ggplot(aes("year", "number", color="sex"))
    + geom_line()
    + expand_limits(y=0)
    + facet_wrap("~ field")
)

In [ ]:
from siuba.dply.forcats import fct_reorder


example = (    recipients_year_field_sex
    >> spread(_.sex, _.number)
    >> mutate(pct_male=_.Male / _.All)
    >> filter(_.broad_field == "Engineering")
)

fct_reorder(example.field, -example.pct_male)

In [ ]:
(
    recipients_year_field_sex
    # this is spread(_.type, _.value) in pandas --------
    # unstack to row-index: field, type. multi-column-index: value[Number, Percent]
    .pipe(_.set_index(["broad_field", "field", "year", "sex"]).unstack(level = -1))
    # get rid of multi-column-index
    .pipe(_.reset_index(col_level = 1).droplevel(None, axis = 1))
    # make it so our columns don't have a "name"
    .pipe(_.rename_axis(columns = None))
    # rest of pipe ----
    .assign(pct_male=_.Male / _.All)
    .loc[_.broad_field == "Engineering"]
    .assign(field = fct_reorder(_.field, -_.pct_male))
    >> ggplot(aes("year", "pct_male", color="field"))
    + geom_line()
    + scale_y_continuous(labels=lambda l: ["{:.0f}%".format(v * 100) for v in l])
    + labs(
        x="Year",
        y="% of PhD recipients reporting as male",
        color="Major field",
        title="Breakdown by sex over time within Engineering major fields",
    )
)

In [ ]:
(
    recipients_year_field_sex
    >> spread(_.sex, _.number)
    >> mutate(pct_male=_.Male / _.All)
    >> filter(_.broad_field == "Humanities and arts")
    >> mutate(field = fct_reorder(_.field, -_.pct_male))
    >> ggplot(aes("year", "pct_male", color="field"))
    + geom_line()
    + scale_y_continuous(labels=lambda l: ["{:.0f}%".format(v * 100) for v in l])
    + labs(
        x="Year",
        y="% of PhD recipients reporting as male",
        color="Major field",
        title="Breakdown by sex over time within Humanities & Arts major fields",
    )
)

In [ ]:
(
    recipients_year_field_sex
    >> spread(_.sex, _.number)
    >> mutate(pct_male=_.Male / _.All)
    >> filter(_.broad_field == "Education")
    >> mutate(field = fct_reorder(_.field, -_.pct_male))
    >> ggplot(aes("year", "pct_male", color="field"))
    + geom_line()
    + scale_y_continuous(labels=lambda l: ["{:.0f}%".format(v * 100) for v in l])
    + labs(
        x="Year",
        y="% of PhD recipients reporting as male",
        color="Major field",
        title="Breakdown by sex over time within Education major fields",
    )
)